# **Predicting Tacos Notes**

***
The goal of this project is to analyze food data. We will use a dataset about burritos. It comes with feature characteristics about the customers, composition of the burrito, locations *etc.*. The dataset is available on GoogleSheets and can be downloaded there at a csv format before uploading it on DataBricks:
https://docs.google.com/spreadsheets/d/18HkrklYz1bKpDLeL-kaMrGjAhUM6LeJMIACwEljCgaw/edit#gid=1703829449


First of all the aim will be to predict the note given by users to burritos they ate. It involves building a model to do so given explanatory variables originally implemented in the dataset but also with new ones created while doing data preparation. Besides, we want to do this project along with a big data framework. Hence, treatments will be done using Spark coding language.

In a second part the aim will be to predict a convenient burrito composition given user information. Such techniques could be used to customize commercials sent to customers by email ads or on online food delivering platforms. 
***

## Packages

In [3]:
#pyspark.sql functions: when, count, col etc.
from pyspark.sql import functions as fn
from pyspark.sql import Window

#ml lib tools:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Tokenizer, StopWordsRemover
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml.regression import LinearRegression

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.stat import Correlation
from pyspark.mllib.stat import Statistics
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer, IndexToString

#Array, mathematics operations:
import numpy as np
import matplotlib.pyplot as plt

#Dataframes:
import pandas as pd

#Sentiment analysis:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## 1 Data Import

In [5]:
#file path
df_path = '/FileStore/tables/Burrito___10D-6a08a.csv'

#Loading the Burrito csv file
df = spark.read.format('csv').options(header='true', inferSchema='true').load(df_path)

Let's have a look at the whole database:

In [7]:
display(df)

Location,Burrito,Date,Neighborhood,Address,URL,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa22,Synergy,Wrap,overall,Rec,Reviewer,Notes,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
Donato's taco shop,California,1/18/2016,Miramar,6780 Miramar Rd,http://donatostacoshop.net/,3.5,4.2,null,6.49,3.0,null,null,null,null,null,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,3.8,null,Scott,good fries: 4/5,null,null,x,x,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Oscar's Mexican food,California,1/24/2016,San Marcos,225 S Rancho Santa Fe Rd,http://www.yelp.com/biz/oscars-mexican-food-san-marcos,3.5,3.3,null,5.45,3.5,null,null,null,null,null,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,3.0,null,Scott,Fries: 3/5; too little meat,null,null,x,x,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Oscar's Mexican food,Carnitas,1/24/2016,null,null,null,null,null,null,4.85,1.5,null,null,null,null,null,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,3.0,null,Emily,null,null,null,null,x,x,null,null,null,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Oscar's Mexican food,Carne asada,1/24/2016,null,null,null,null,null,null,5.25,2.0,null,null,null,null,null,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,3.75,null,Ricardo,Go to average burrito place like Rigoberto's in la jolla; Terrible tamales,null,null,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Pollos Maria,California,1/27/2016,Carlsbad,3055 Harding St,http://pollosmaria.com/,4.0,3.8,x,6.59,4.0,null,null,null,null,null,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,4.2,null,Scott,null,null,null,x,x,null,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Pollos Maria,combo chicken,1/28/2016,null,null,null,null,null,null,6.99,4.0,null,null,null,null,null,3.0,4.0,5.0,3.5,2.5,2.5,2.5,4.0,1.0,3.2,null,Emily,null,null,null,null,null,x,x,null,x,null,x,null,null,x,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Nico's Taco Shop,California,1/30/2016,Carmel Valley,3860 Valley Centre Dr #404,http://www.yelp.com/biz/nicos-taco-shop-san-diego,3.0,2.9,null,7.19,1.5,null,null,null,null,null,2.0,3.0,3.0,2.0,2.5,2.5,null,2.0,3.0,2.6,null,Scott,not fries. big potatoes,null,null,x,null,null,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Nico's Taco Shop,Carnitas,1/30/2016,null,null,null,null,null,null,6.99,4.0,null,null,null,null,null,2.5,3.0,3.0,2.5,3.0,3.5,null,2.5,3.0,3.0,null,Emily,null,null,null,null,x,x,null,null,null,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Los Primos Mexican Food,Monster California,2/1/2016,UTC,7770 Regents Rd,http://www.primosmex.com/,3.0,3.7,x,9.25,3.5,null,null,null,null,null,2.0,4.5,4.5,3.5,1.5,3.0,3.5,4.0,2.0,3.9,null,Scott,this tasted really bad leftover. not included in rating,null,null,x,x,x,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
JV's Mexican Food,Carne Asada,2/6/2016,Morena,1112 Morena Blvd,http://jvsmexfood.com/,4.0,4.1,null,6.25,3.5,n

#### Dataset dimensions

In [9]:
#Dataset dimensions:
N = df.count() #N observations
P = len(df.columns) #P columns
print(f'There are {N} observations and {P} columns in the dataset.')

There are 423 observations and 66 columns in the dataset.

## 2 Data Cleaning

### 2.1 Observations ID

The dataset does not contain the id for each observation, we can do so as followed:

In [13]:
df = df.withColumn('id', fn.monotonically_increasing_id())

### 2.2 Handling missing values

Before computing preliminary descriptive statistics and reformatting variable, let's display the number of missing values per column:

In [16]:
na_per_column = df.select([fn.count(fn.when(df[c].isNull(), c)).alias(c) for c in df.columns])
display(na_per_column)

Location,Burrito,Date,Neighborhood,Address,URL,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa22,Synergy,Wrap,overall,Rec,Reviewer,Notes,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,id
0,0,0,331,335,336,336,336,397,7,3,401,401,139,141,141,0,20,14,3,9,2,25,2,3,2,190,1,277,390,416,243,264,268,263,295,331,372,402,402,417,387,388,412,416,416,422,415,385,416,408,406,419,416,421,419,419,422,423,418,420,420,421,410,420,422,0


For the rest of the project we have to keep in mind that the missing value format is expressed as 'null', hence isNull() or isNotNull() functions will be used instead of isna().

#### 2.2.1 `Neighborhood`, `Adress`, `Yelp` & `Google`

We noticed the columns `Neighborhood`, `Adress`, `Yelp`, `Google` only contain value for the first time a shop is rated in the dataset. We want to fill the NAs according to the `Location`, which is the **join key**. Hence we will filter the dataset accross distinct non null values then merge both data frames according to the `Location`.

In [20]:
for c in ['Neighborhood', 'Yelp', 'Google']:
  #Create a temporary dataframe where only distinct non value for 'Location' and c are stored:
  temp = df.select("Location", c).distinct()
  
  #filtering on non null values:
  temp = temp.filter(temp[c].isNotNull())
  
  #Merging with the original dataset df:
  df = df.drop(c).join(temp,'Location', 'left')
  
#Display the results:
display(df.limit(5))

Location,Burrito,Date,Address,URL,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa22,Synergy,Wrap,overall,Rec,Reviewer,Notes,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,id,Neighborhood,Yelp,Google
Lolita's Taco shop,Carne asada,5/27/2016,null,null,null,6.25,4.0,null,null,15.0,20.5,0.5,1.5,2.5,3.0,4.5,4.0,2.0,2.75,3.75,3.0,2.6,No,Scott,Way too small,null,null,X,X,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,120,null,null,null
Lolita's Taco shop,Shrimp,5/27/2016,null,null,null,8.95,3.5,null,null,18.5,25.0,0.92,3.0,3.0,4.0,4.0,3.5,3.5,2.5,4.0,0.5,3.0,Yes,Emily,null,null,null,null,X,null,null,null,null,null,null,X,null,null,null,X,null,null,null,null,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,121,null,null,null
Lolita's Taco shop,Bean and cheese,5/27/2016,null,null,null,3.5,1.0,null,null,16.0,20.0,0.51,3.5,4.5,null,3.0,null,4.0,3.0,2.5,4.0,2.7,No,Scott,null,null,null,null,null,null,x,null,null,null,null,null,null,null,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,122,null,null,null
Tacos La Bala,Pastor,9/5/2016,5800 Bellaire Blvd,tacoslabala.com,null,4.99,3.5,null,null,null,null,null,2.5,2.5,4.0,2.5,1.0,1.0,2.0,4.0,0.0,2.5,No,Scott,Everything is in spanish. Get a taco or sope,null,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,222,Houston,4.5,4.1
Tacos La Bala,Barbacoa,9/5/2016,null,null,null,4.99,2.5,null,null,16.5,22.0,0.64,2.0,2.5,3.5,2.0,3.0,2.5,2.0,2.5,2.5,2.4,No,Emily,"Burritos cut in half, felt like a wrap instead of a burrito",null,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,223,Houston,4.5,4.1


Let's check if there are still missing values:

In [22]:
display(df.select([fn.count(fn.when(df[c].isNull(), c)).alias(c) for c in ['Neighborhood', 'Yelp', 'Google']]))

Neighborhood,Yelp,Google
63,66,66


There are still missing values, but we will handle them by setting them into a class when we will do the data preparation.

#### 2.2.2 `Cost` & `Hunger`

These two variables do not contain so much NAs, hence we can replace them by imputing their mean value.

For both columns, let's compute the mean over the column values not containing any null:

1. We have to first filter over df where it's column `Cost` isn't null
2. Compute the average
3. Collect the value with the first() operation and retain it by adding [0] to keep the value within it
4. use numpy.round() function to retain only 2 decimals

In [26]:
for c in ['Cost', 'Hunger']:
  #Calculating the mean and round it to 2 decimals:
  mean = np.round(df.where(df[c].isNotNull()).agg({c : 'avg'}).first()[0],2)
  print(f"The average value for column '{c}' is about {mean}")

  #Create the decision rule; according to a null value condition either we inpute the value either the computed mean
  new = fn.when(df[c].isNull(), mean)\
               .otherwise(df[c])

  #We apply the decision rule and inpute it to actual column:
  df = df.withColumn(c, new)

The average value for column 'Cost' is about 7.07
The average value for column 'Hunger' is about 3.5

#### 2.2.3 Burrito's composition dummies

There are several columns indicating whether the burritos contains a food or not. As they are filled with "x" indicating the food is present, or "null" indicating the food is absent, we will display those variables in a better format. The column `Chips` is located apart from the others so we will treat it right after.

In [29]:
#Select the food variable names
columns_food = df.columns[28:63]

#Display:
display(df.select(columns_food).limit(5))

Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
X,X,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,X,null,null,null,null,null,null,X,null,null,null,X,null,null,null,null,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,x,null,null,null,null,null,null,null,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


We can do a loop over those variables in order to re-set them in a good format:

In [31]:
#loop over the columns in `columns_food`:
for c in columns_food:
  #Decision rule: 0 if null, 1 otherwise
  binary = fn.when(df[c].isNull(), 0)\
             .otherwise(1)
  
  #update the corresponding column:
  df = df.withColumn(c,binary)

Let's do it on `Chips`:

In [33]:
Chips = fn.when(df['Chips'].isNull(), 0)\
          .otherwise(1)
df = df.withColumn('Chips', Chips)

Checking it has been done correctly:

In [35]:
#Display:
display(df.select(columns_food).limit(5))

Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### 2.2.4 `Notes`

Some notes are missing, but it is compulsory to have a string value in order to do text or sentiment analysis on it:

In [38]:
#Decision rule, we set "Neutral" whenever the value is missing:
notes = fn.when(df["Notes"].isNotNull(), df["Notes"])\
          .otherwise("Neutral")

#We apply it:
df = df.withColumn("Notes", notes)

#Display of 5 rows:
df.select("Notes").show(5)

+--------------------+
 Notes|
+--------------------+
 Way too small|
 Neutral|
 Neutral|
Everything is in ...|
Burritos cut in h...|
+--------------------+
only showing top 5 rows

#### 2.2.5 `overall`

`overall`, the target variable, contains 2 missing, values, as we can't know the real values for them, we can discard the corresponding rows and reset the number of observations.

In [41]:
#Discarding rows for overall is missing:
df = df.filter(df['overall'].isNotNull())

#Reset the number of observations:
N = df.count() #N observations
print(f'There are {N} observations.')

There are 421 observations.

### 2.3 Dropping variables

As we noticed that `Neighborhood`,`URL` and `Adress` were filled the same way in the dataset, we do not want repetitive information within it, so we can delete those columns.

Furthermore, we can delete it **endogenous variables** such like variables linked to the dependent variable `overall` itself: `Tortilla`, `Meat`, `Fillings`, `Meat:filling`, `Uniformity`, `Salsa22`, `Synergy`, `Wrap`

Other features such as `Rec`, `Date`, `Temp` and `Address` might also be discarded:

In [44]:
df = df.drop('URL','Adress', 'Mass (g)', 'Density (g/ML)', 'Unreliable', 'NonSD', 'Tortilla', 'Meat',
             'Fillings', 'Meat:filling', 'Uniformity', 'Salsa22', 'Synergy', 'Wrap', 'Rec', 'Address',
             'Temp', 'Date')

## 3 Data preparation

The data is now cleaned of missing values except the ones we left which will be used in the data preparation part to create others variables.

### 3.1 One-hot encoding

#### 3.1.1 `Burrito`

Let's display some burrito names:

In [50]:
df.select('Burrito').distinct().show(5)

+------------------+
 Burrito|
+------------------+
 Colimas burrito|
 Shrimp|
 Azteca|
 Spicy a la Diabla|
Bitchin California|
+------------------+
only showing top 5 rows

In [51]:
#We delete spaces at the end of the word:
df = df.withColumn('Burrito', fn.trim(df.Burrito))

#We set it to lowercase:
df = df.withColumn('Burrito', fn.lower(df.Burrito))

#Showing some districs to check operations have been done correctly:
df.select('Burrito').distinct().show(5)

#Number of different burritos:
n_burrito = neighborhood = df.select('Burrito').distinct().count()
print(f'There are {n_burrito} different burritos.')

+------------------+
 Burrito|
+------------------+
 mauna lani|
 quesaburro|
monster california|
 local|
 deborah's special|
+------------------+
only showing top 5 rows

There are 116 different burritos.

Let's display how many times a Burrito appears in the dataset:

In [53]:
count_burrito = df.groupBy('Burrito').agg(fn.count('Location').alias('count')).orderBy(("count"), ascending=False)
display(count_burrito)

Burrito,count
california,127
carne asada,35
carnitas,24
california everything,16
local,14
surf & turf,14
al pastor,13
adobada,10
surfin california,9
custom,5


The California burrito in the most represented one. Let's show the burritos which appears more than 5 times:

In [55]:
known_burrito = count_burrito.where(count_burrito['count'] > 5).select('Burrito')
display(known_burrito)

Burrito
california
carne asada
carnitas
california everything
surf & turf
local
al pastor
adobada
surfin california


We can now create a dummy variable for each of these burrito's name:

In [57]:
for burrito in [str(row.Burrito) for row in known_burrito.collect()]:
  dum = fn.when(df['Burrito']==burrito, 1).otherwise(0)
  df = df.withColumn(burrito, dum) #We apply the decision rule
  df = df.withColumnRenamed(burrito,burrito.replace(" ", "_")) #Replace spaces by underscores

We noticed the Burrito named "Surf and Turf" is also named "Surf & Turf":

In [59]:
df = df.withColumnRenamed('surf_&_turf', 'surf_and_turf') #We rename the first column created
dum_surfturf = fn.when(df['Burrito'] == 'surf and turf', 1).otherwise(df['surf_and_turf']) #to get all obs. who ate this burrito
df = df.withColumn('surf_and_turf', dum_surfturf) #we apply the decision rule

In [60]:
## For later on for data analaysis
df = df.withColumn('burrito_forstat', fn.when(df['california']+df['carne_asada']+df['carnitas']+df['local']+df['surf_and_turf']+
                                              df['california_everything']+df['adobada']+
                                              df['al_pastor']+df['surfin_california'] + df['surf_and_turf'] ==1, df.Burrito)\
                                         .otherwise("Special Burrito"))

Observations where none of these dummies are set at 1 are 'special tacos', meaning there are creation from the restaurant or not famous/ more original tacos. There is no dummies to represent these tacos to avoid identification issue for the classification.

Now we can drop the `Burrito` column:

In [62]:
df = df.drop('Burrito')

#### 3.1.2 `Neighborhood`

Let's count the number of different locations and neighborhoods, and do some text cleaning on it:

In [65]:
#We delete spaces at the end of the word:
df = df.withColumn('Neighborhood', fn.trim(df.Neighborhood))

#We set it to lowercase:
df = df.withColumn('Neighborhood', fn.lower(df.Neighborhood))

#Showing some districs to check operations have been done correctly:
df.select('Neighborhood').distinct().show(5)

#Number of different neighborhoods:
n_neighborhood = df.select('Neighborhood').distinct().count()
print(f'There are {n_neighborhood} different neighborhoods.')

+------------+
Neighborhood|
+------------+
 san marcos|
 carlsbad|
 ocean beach|
 new york|
 la jolla|
+------------+
only showing top 5 rows

There are 39 different neighborhoods.

In some cases the neighborhood is missing, we will reshape the `Neighborhood` variable which will display 'No neighborhood' when this one is missing.

In [67]:
df.select('Neighborhood').distinct().show()

+-----------------+
 Neighborhood|
+-----------------+
 san marcos|
 carlsbad|
 ocean beach|
 new york|
 la jolla|
 pacific beach|
 university city|
 santa monica|
 del mar|
 mission beach|
 null|
 north park|
 south park|
la jolla village|
 linda vista|
 sorrento valley|
 carmel valley|
 webster|
 utc|
 hillcrest|
+-----------------+
only showing top 20 rows

In [68]:
count_neighborhood = df.groupBy('Neighborhood').agg(fn.count('Location').alias('count')).orderBy(("count"), ascending=False)
display(count_neighborhood)

Neighborhood,count
null,63
miramar,42
utc,42
north park,31
clairemont,29
linda vista,27
la jolla,23
encinitas,23
carlsbad,14
oceanside,14


In [69]:
known_neighborhood = count_neighborhood.where(count_neighborhood['count'] > 5).select('Neighborhood')
display(known_neighborhood)

Neighborhood
null
utc
miramar
north park
clairemont
linda vista
la jolla
encinitas
carlsbad
oceanside


In [70]:
for neigh in [str(row.Neighborhood) for row in known_neighborhood.collect()]:
  dum = fn.when(df['Neighborhood']==neigh, 1).otherwise(0)
  df = df.withColumn(neigh, dum)
  df = df.withColumnRenamed(neigh,neigh.replace(" ", "_"))
df.select('utc').show(10)

+---+
utc|
+---+
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
+---+
only showing top 10 rows

#### 3.1.3 `Location`

Let's do the same procedure for `Location`:

In [73]:
#We delete spaces at the end of the word:
df = df.withColumn('Location', fn.trim(df.Location))

#We set it to lowercase:
df = df.withColumn('Location', fn.lower(df.Location))

#Showing some districs to check operations have been done correctly:
df.select('Location').distinct().show(5)

#Number of different neighborhoods:
n_location = df.select('Location').distinct().count()
print(f'There are {n_location} different location.')

+--------------------+
 Location|
+--------------------+
 goody's|
caliente mexican ...|
 tacos por favor|
 king burrito|
 lolita's taco shop|
+--------------------+
only showing top 5 rows

There are 99 different location.

In [74]:
count_location = df.groupBy('Location').agg(fn.count('Location').alias('count')).orderBy(("count"), ascending=False)
display(count_location)

In [75]:
known_location = count_location.where(count_location['count'] > 5).select('Location')
display(known_location)

In [76]:
for location in [str(row.Location) for row in known_location.collect()]:
  dum = fn.when(df['Location']==location, 1).otherwise(0)
  df = df.withColumn(location, dum)
  df = df.withColumnRenamed(location,location.replace(" ", "_").replace("'", "_"))

#### 3.1.3 `Reviewer`

In [78]:
df = df.withColumn("Reviewer", fn.trim(df.Reviewer))
df = df.withColumn("Reviewer", fn.lower(df.Reviewer))
avg_reviewer =df.groupBy('Reviewer').agg(fn.mean(df.overall).alias('avg_grade'),
                                         fn.count(df.id).alias("n"))
print('There is ' + str(avg_reviewer.count()) + ' reviewers in the dataset')
avg_reviewer.show(5)

In [79]:
alpha=5
meanrev = df.agg({"overall" : 'avg'}).first()[0]
smoothed = fn.udf(lambda x,n: (n*x + meanrev*alpha)/(n + alpha))
avg_reviewer = avg_reviewer.withColumn("avg_smooth", smoothed(avg_reviewer.avg_grade, avg_reviewer.n))
avg_reviewer.show(5)

In [80]:
mean = avg_reviewer.agg({"avg_smooth" : 'avg'}).first()[0]
stdev = avg_reviewer.agg({"avg_smooth" : 'std'}).first()[0]
correction = fn.udf(lambda x: (x-mean))
avg_reviewer = avg_reviewer.withColumn("avg_grade_scaled",correction(avg_reviewer.avg_smooth))
avg_reviewer.show(5)  

In [81]:
avg_reviewer = avg_reviewer.drop("avg_grade")
avg_reviewer = avg_reviewer.drop("avg_smooth")
avg_reviewer = avg_reviewer.drop("n")

### 3.2 Discretization

- We will discretize continuous variables according to quartiles because some of them contain too much NAs, hence a class will be dedicated to them.
- In order to avoid collinearity we will only keep the NA class, the first and second quartile.

#### 3.2.1 `Length`

To get an idea of the distribution of the variable `Length`:

In [86]:
print('Minimum: ', df.select(fn.min(df['Length'])).collect()[0][0])
print('First quartile: ', df.where(df['Length'].isNotNull()).approxQuantile('Length', [0.25], 0.25)[0])
print('Median: ',df.where(df['Length'].isNotNull()).approxQuantile('Length', [0.5], 0.25)[0])
print('Third quartile: ',df.where(df['Length'].isNotNull()).approxQuantile('Length', [0.75], 0.25)[0])
print('Maximum: ',df.select(fn.max(df['Length'])).collect()[0][0])

We can create dummys according to the quartiles and missing values:

In [88]:
#NA:
length_na = fn.when(df['Length'].isNull(), 1).otherwise(0)
df = df.withColumn('length_na', length_na)

#1st quartile:
length_25 = fn.when(df['Length'] <= 15, 1).otherwise(0)
df = df.withColumn('length_25', length_25)

#2nd quartile:
length_50 = fn.when( (df['Length'] > 15) & (df['Length'] < 22), 1).otherwise(0)
df = df.withColumn('length_50', length_50)

#### 3.2.2 `Circum`

To get an idea of the distribution of the variable `Circum`:

In [91]:
print("Minimum: ", df.select(fn.min(df['Circum'])).collect()[0][0])
print("First quartile: ", df.where(df['Circum'].isNotNull()).approxQuantile("Circum", [0.25], 0.25)[0])
print("Median: ", df.where(df['Circum'].isNotNull()).approxQuantile("Circum", [0.5], 0.25)[0])
print("Third quartile: ", df.where(df['Circum'].isNotNull()).approxQuantile("Circum", [0.75], 0.25)[0])
print("Maximum: ", df.select(fn.max(df['Circum'])).collect()[0][0])

We can create dummys according to the quartiles and missing values:

In [93]:
#NA:
circum_na = fn.when(df['Circum'].isNull(), 1).otherwise(0)
df = df.withColumn('circum_na', circum_na)

#1st quartile:
circum_25 = fn.when(df['Circum'] <= 17, 1).otherwise(0)
df = df.withColumn('circum_25', circum_25)

#2nd quartile:
circum_50 = fn.when( (df['Circum'] > 17) & (df['Circum'] < 22.5), 1).otherwise(0)
df = df.withColumn('circum_50', circum_50)

#### 3.2.3 `Volume`

To get an idea of the distribution of the variable `Volume`:

In [96]:
print('Minimum: ', df.select(fn.min(df['Volume'])).collect()[0][0])
print('First quartile: ', df.where(df['Volume'].isNotNull()).approxQuantile('Volume', [0.25], 0.25)[0])
print('Median: ', df.where(df['Volume'].isNotNull()).approxQuantile('Volume', [0.5], 0.25)[0])
print('Third quartile: ', df.where(df['Volume'].isNotNull()).approxQuantile('Volume', [0.75], 0.25)[0])
print('Maximum: ', df.select(fn.max(df['Volume'])).collect()[0][0])

We can create dummys according to the quartiles and missing values:

In [98]:
#NA:
volume_na = fn.when(df['Volume'].isNull(), 1).otherwise(0)
df = df.withColumn('volume_na', volume_na)

#1st quartile:
volume_25 = fn.when(df['Volume'] <= 0.4, 1).otherwise(0)
df = df.withColumn('volume_25', volume_25)

#2nd quartile:
volume_50 = fn.when( (df['Volume'] > 0.4) & (df['Volume'] < 0.79), 1).otherwise(0)
df = df.withColumn('volume_50', volume_50)

#### 3.2.4 `Yelp`

In [100]:
print("Minimum: ", df.select(fn.min(df['Yelp'])).collect()[0][0])
print("First quartile: ", df.where(df['Yelp'].isNotNull()).approxQuantile("Yelp", [0.25], 0.25)[0])
print("Median: ", df.where(df['Yelp'].isNotNull()).approxQuantile("Yelp", [0.5], 0.25)[0])
print("Third quartile: ", df.where(df['Yelp'].isNotNull()).approxQuantile("Yelp", [0.75], 0.25)[0])
print("Maximum: ", df.select(fn.max(df['Yelp'])).collect()[0][0])

In [101]:
#NA:
yelp_na = fn.when(df['Yelp'].isNull(), 1).otherwise(0)
df = df.withColumn('yelp_na', yelp_na)

#1st quartile:
yelp_25 = fn.when(df['Yelp'] <= 2.5, 1).otherwise(0)
df = df.withColumn('yelp_25', yelp_25)

#2nd quartile:
yelp_50 = fn.when( (df['Yelp'] > 2.5) & (df['Yelp'] < 4.2), 1).otherwise(0)
df = df.withColumn('yelp_50', yelp_50)

#### 3.2.5 `Google`

In [103]:
print("Minimum: ", df.select(fn.min(df['Google'])).collect()[0][0])
print("First quartile: ", df.where(df['Google'].isNotNull()).approxQuantile("Google", [0.25], 0.25)[0])
print("Median: ", df.where(df['Google'].isNotNull()).approxQuantile("Google", [0.5], 0.25)[0])
print("Third quartile: ", df.where(df['Google'].isNotNull()).approxQuantile("Google", [0.75], 0.25)[0])
print("Maximum: ", df.select(fn.max(df['Google'])).collect()[0][0])

In [104]:
#NA:
google_na = fn.when(df['Google'].isNull(), 1).otherwise(0)
df = df.withColumn('google_na', google_na)

#1st quartile:
google_25 = fn.when(df['Google'] <= 2.9, 1).otherwise(0)
df = df.withColumn('google_25', google_25)

#2nd quartile:
google_50 = fn.when( (df['Google'] > 4.3) & (df['Google'] < 5), 1).otherwise(0)
df = df.withColumn('google_50', google_50)

### 3.3 Sentiment Analysis / Opinion Mining

You can find below some example of the text on which we are going to execute sentiment analysis:

In [107]:
df.select('Notes').show(10)

The output of the analyzer comes from the Vader package, it outputs scores about how much the notes is positive, negative or neutral. There are called *pos, neg* and *neu* respectively.

$$ pos, neg, neu  \in \[0,1\]  $$ 

It also provides a compound score of it: $$ compound \in [-1,1] $$

**NB:** we keep the "id" column values in the rdds because it will be used as schema further in order to comeback to a dataframe format.

In [110]:
#Sentiment analyzer
analyser = SentimentIntensityAnalyzer()

#Get sentiment scores: negativity, positivity, neutrality and compound 
def get_sentiment(x):
    return analyser.polarity_scores(x)

#Storing "Notes" as rdd
#lambda function: rank 0 for the id, 1 for the Notes
notes_rdd = df.select('id', 'Notes').rdd.map(lambda x : (x[0], x[1]))

#We map the function get_sentiment over the rdd we created:
#lambda function: rank 0 to keep the id, apply get_sentiment on rank 1
sentiment_rdd = notes_rdd.map(lambda x: (x[0],get_sentiment(x[1])))
sentiment_rdd.take(5)

Now we can extract positivity, negativity and compound and set them into an rdd. We do not use neutrality as:

$$neg + neu + post = 1$$

We want to avoid for collinearity between columns.

In [112]:
#Reshaping the rdd
sentiment_rdd = sentiment_rdd.map(lambda x: (x[0], x[1]['neg'], x[1]['pos'], x[1]['compound']))

sentiment_rdd.take(5)

Here is a first example, when the comment is missing:

In [114]:
#we take the second individual for the sake of the example
i = 2
print(notes_rdd.take(i)[i-1][1])
print('Negativity:', sentiment_rdd.take(i)[i-1][1])
print('Positivity:', sentiment_rdd.take(i)[i-1][2])
print('Compound:', sentiment_rdd.take(i)[i-1][3])

As we can, the value we set for missing values outputs neither negativity nor positivity. Indeed it output neutrality at level 1.

Let's show another example:

In [116]:
i = 5
print(notes_rdd.take(i)[i-1][1])
print('Negativity:', sentiment_rdd.take(i)[i-1][1])
print('Positivity:', sentiment_rdd.take(i)[i-1][2])
print('Compound:', sentiment_rdd.take(i)[i-1][3])

The example above demonstrates pretty how much complexity can emanate from a note.

Checking sentiment_rdd is at the right size:

In [118]:
#Check number of rows:
assert sentiment_rdd.count() == df.count()

No error should come from the code above.

Let's create a dataframe containing sentiment information we collected:

In [120]:
sentiment_df = sentiment_rdd.toDF(['id', 'neg', 'pos', 'compound'])
sentiment_df.show(10)

We can now join sentiment_df with the original raw data df:

In [122]:
#Join:
df = df.join(sentiment_df, 'id', 'left')

### 3.4 Term Frequency-Inverse Document Frequency (TF-IDF)

In [124]:
#We filter our analysis on real comments:
notes = df.where(df['Notes']!='Neutral').select("id", "Notes")

Firstly, we need to tokenize the comments:

In [126]:
tokenizer = Tokenizer(inputCol ='Notes', outputCol='words_token')
notes = tokenizer.transform(notes).select('id', 'words_token')
notes.show(5)

Secondly, we need to delete stopwords which don't give any interesting informations within the comment:

In [128]:
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
notes = remover.transform(notes).select('id', 'words_clean')
notes.show(5)

In [129]:
notes_byword = notes.select(fn.explode(notes.words_clean).alias('word'), 'id')
notes_byword.show(30)

Here, we delete the ponctuation to be able to recognize each word:

In [131]:
notes_byword = notes_byword.select(fn.regexp_replace(notes_byword.word, r'[?.!,;]', '').alias('word'), 'id')
notes_byword.show(10)

Here, we group words to get the frequency within the comment and the frequency of comment where the word appears in it.

In [133]:
tf_word = notes_byword.groupBy('id', 'word').agg(fn.count('id').alias('tf'))
df_word = notes_byword.groupBy('word').agg(fn.countDistinct('id').alias('df'))
nb_word = notes_byword.groupBy('id').agg(fn.count('word').alias('nb'))
tf_word.show(5)
df_word.show(5)
nb_word.show(5)
print('The total number of words is of '+ str(df_word.count()) + '.')

In [134]:
df_word.orderBy(("df"), ascending=False).show(100)

In [135]:
n_max = notes.count()/2
n_min = 10 
df_word = df_word.where(df_word.df<n_max)
df_word = df_word.where(df_word.df>=n_min)
df_word.show()
tf_word = tf_word.join(df_word,'word', 'inner')
tf_word = tf_word.drop('df')
tf_word.show(10)

There are only 10 words left and they doen't seem informative by themselve. Combinaison of them can be interesting for analysis ( as 'not good') but alone it is only words about ingredients or feelings, the sentiment analyzer already captured this part and we also information about food inside the burrito. So we decided not to use TF-IDF.

Hence we can now drop the variable `Notes`:

In [137]:
#Drop:
df = df.drop('Notes')

### 3.4 Normalisation of continuous variables

In order to treat data properly, the prediction model performs better with normalized data. Hence we will use the StandardScaler normalizer to do so, on a given variable \\(var\\), its mean \\(\overline{var}\\) and its standard deviation \\(\mathrm{std}(var)\\), we are going to normalize each observation as followed:

$$ \forall i \in \\{1,...,N\\}: scaled\\_var_i = \dfrac{var_i - \overline{var}}{\mathrm{std}(var)}$$

In [140]:
continuous_var = ['Cost', 'Hunger', 'neg', 'pos', 'compound']

In [141]:
for var in continuous_var: 
  mean = df.agg({var : 'avg'}).first()[0]
  stdev = df.agg({var : 'std'}).first()[0]
  stdz = fn.udf(lambda x: (x-mean)/stdev)
  df = df.withColumn(var,stdz(df[var]))

In [142]:
df.select(continuous_var).show(5)

### 3.5 Reshaping the target variable: `overall`

A logical decision would be to create a dummy which decision is ruled whether the target variable is above or below its median, let's compute the median:

In [145]:
#Computing the median with the approxQuantile function
#1st term: variable | 2nd term : quantile | 3rd term : relative error
median_overall = df.approxQuantile('overall', [0.5], 0.25)[0]

print('The median is about', median_overall, 'for the target variable.')

The target variable `overall` is originally continuous, we will reshape it into a dummy variable called `good_grade` according to the following decision rule:
$$
\overline{overall} = \displaystyle \dfrac{1}{n} \sum_{i=1}^{N} overall_i
$$
  
$$ \forall i \in \\{1,\ldots,N\\} ~: goodgrade_i = \begin{cases} 1 ~ \mathrm{if} ~ overall_i > \overline{overall}
\cr 0 ~ \mathrm{otherwise} \end{cases} $$  

Firstly, we shall compute \\(\overline{overall}\\) :

In [147]:
#Calculating the mean and round it to 2 decimals:
mean_overall = np.round(df.agg({'overall' : 'avg'}).first()[0],2)
print(mean_overall)

Now we are able to apply the decision rule:

In [149]:
#Create the decision rule; according to the overall grade, we will input the variable good_grade to 1 if it is higher than the mean, 0 otherwise
good_grade = fn.when(df['overall'] > mean_overall, 1)\
               .otherwise(0)

#We apply the decision rule and inpute it to actual column:
df = df.withColumn('good_grade', good_grade)

In [150]:
#To check the decision rule has been applied
print(df.select('overall', 'good_grade').show(10))

#Dropping the original target variable:
df_withall = df
df = df.drop('overall')

## 4 Data Analysis

### 4.1 Descriptive statistics

Let's define the following function, it will compute the following statistics for a variable \\(var\\) according to the group: \\(goodgrade = k ~, k \in \\{0,1\\}\\)

1- \\(\mathrm{count}(goodgrade = k) = |i: goodgrade_i = k|\\)

2- \\(\mathrm{mean}(var) = \overline{var} = \displaystyle \dfrac{1}{N} \sum_{i = 1}^{N} var_i\\)

3- \\(\mathrm{std}(var) = \sqrt{\displaystyle \dfrac{1}{N} \sum_{i = 1}^{N} (var_i - \overline{var})^2}\\)

4- \\( \mathrm{min}(var)\\)

5- \\( \mathrm{quantile}(var)_p, p \in \\{0.25, 0.50, 0.75\\} = inf\\{var \in \mathbb{R}, p \le F(var)\\} ~ \mathrm{and} ~ F_X(x) = \mathbb{P}(X \le x) = p \\)

6- \\(\mathrm{max}(var) \\)

In [154]:
def descriptive(var):
  print(f'Statistics for the variable {var}:')
  df.groupby("good_grade")\
    .agg(fn.count(var).alias('count'),
                     fn.round(fn.mean(var),2).alias('mean'),
                     fn.round(fn.stddev(var),2).alias('std'),
                     fn.round(fn.min(var),2).alias('min'),
                     fn.round(fn.expr(f'percentile({var}, array(0.25))')[0],2).alias('%25'), 
                     fn.round(fn.expr(f'percentile({var}, array(0.5))')[0],2).alias('%50'),
                     fn.round(fn.expr(f'percentile({var}, array(0.75))')[0],2).alias('%75'),
                     fn.round(fn.max(var),2).alias('max'))\
    .show()

Let's run it on the following continuous features:

In [156]:
#list:
continuous_var = ['Length', 'Circum', 'Volume', 'Cost', 'Google', 'Yelp', 'neg', 'pos', 'compound']

#Run
for var in continuous_var:
  descriptive(var)

md Counterintuitively, the mean of the variable `pos` is higher for the \\( goodgrade = 0 \\) class, however the standard deviation relatively high so we cannot assert anything from those results.

We can now drop some of the continuous variables as we already created categories based on their quartiles and because the analysis is finished:

In [158]:
df = df.drop('Length', 'Circum', 'Volume', 'Google', 'Yelp')

We can also display the proportion of observations such that \\(goodgrade = 1 \\) per neighborhood as well as the number of different locations:

In [160]:
neighbh = df.groupBy(df.Neighborhood)\
            .agg(fn.mean(df.good_grade).alias('Proportion_of_good_grades'),
                 fn.count(df.id).alias('Number_of_tacos_evaluated'),
                 fn.countDistinct(df.Location).alias("Number_of_restaurant_concerned"))\
            .orderBy(("Proportion_of_good_grades"), ascending=False)
display(neighbh)

Besides, it is also possible to do it per reviewer:

In [162]:
reviwr = df.groupBy(df.Reviewer)\
           .agg(fn.mean(df.good_grade).alias("Proportion_of_good_grades"),
                fn.count(df.id).alias("Number_of_tacos_evaluated"))\
           .orderBy(("Proportion_of_good_grades"), ascending=False)

display(reviwr)

We can now drop those variables:

In [164]:
df = df.drop('Neighborhood', 'Reviewer', 'Location')

### 4.2 Visual analysis

Let's focus our analysis on the type of burrito bought by customers, depending on they have a good grade (\\(goodgrade = 1 \\)) or not (\\(goodgrade = 0 \\)) :

In [167]:
#Counting the number of burrito per type for the good grade class:
good_gradeDF = df.where(df.good_grade==1)\
                 .groupBy(df.burrito_forstat)\
                 .agg(fn.count('id').alias('nb_burrito'))\
                 .select('nb_burrito')\
                 .orderBy(("burrito_forstat"), ascending=True)
#Setting it in a list:
list_L1 = [int(i.nb_burrito) for i in good_gradeDF.collect()]

#Counting the number of burrito per type for the good grade class:
bad_gradeDF = df.where(df.good_grade==0)\
                .groupBy(df.burrito_forstat)\
                .agg(fn.count('id').alias('nb_burrito'))\
                .select('nb_burrito')\
                .orderBy(("burrito_forstat"), ascending=True)
#Setting it in a list:
list_L0 = [int(i.nb_burrito) for i in bad_gradeDF.collect()]

#Create the different modalities
modalities = df.select('burrito_forstat').distinct().orderBy(("burrito_forstat"), ascending=True)
modalities =  [str(i.burrito_forstat) for i in modalities.collect()]

Let's display on a bar plot those figures:

In [169]:
#Required list for the totals:
totals = [i+j for i,j in zip(list_L1,list_L0)]
list_L1p = [i / j * 100 for i,j in zip(list_L1, totals)]
list_L0p = [i / j * 100 for i,j in zip(list_L0, totals)]
p=range(len(list_L1)) 

#Plot options:
barWidth = 0.7
plt.figure(figsize=(12,5))
plt.bar(p, list_L1p, color='#C7CF00', edgecolor='white', width=barWidth)
plt.bar(p, list_L0p, bottom=list_L1p, color='crimson', edgecolor='white', width=barWidth)
plt.xlabel('Type of burrito')
plt.ylabel('Proportion of good and bad grades')
plt.xticks(p, modalities, fontsize='small', size=6)
plt.gca().legend(('Good','Bad'))
display(plt.show())

We can do the exact same analysis based on the food inside the burritos, let's pick some of them: `beef`, `chicken`, `shrimp`, `fish`.

In [171]:
#Lists for the good grades:
list_beef = [int(i.nb) for i in df.where(df.Beef==1).select('good_grade').agg(fn.sum('good_grade').alias('nb')).collect()]
list_chicken = [int(i.nb) for i in df.where(df.Chicken==1).select('good_grade').agg(fn.sum('good_grade').alias('nb')).collect()]
list_shrimp = [int(i.nb) for i in df.where(df.Shrimp==1).select('good_grade').agg(fn.sum('good_grade').alias('nb')).collect()]
list_fish = [int(i.nb) for i in df.where(df.Fish==1).select('good_grade').agg(fn.sum('good_grade').alias('nb')).collect()]
#Gathering them all:
list_L1 = [list_beef[0], list_chicken[0], list_shrimp[0], list_fish[0]]

#Lists for the bad grades:
list_beef0 = [int(i.nb) for i in df.where(df.Beef==1).where(df.good_grade==0).agg(fn.count('id').alias('nb')).collect()]
list_chicken0 = [int(i.nb) for i in df.where(df.Chicken==1).where(df.good_grade==0).agg(fn.count('id').alias('nb')).collect()]
list_shrimp0 = [int(i.nb) for i in df.where(df.Shrimp==1).where(df.good_grade==0).agg(fn.count('id').alias('nb')).collect()]
list_fish0 = [int(i.nb) for i in df.where(df.Fish==1).where(df.good_grade==0).agg(fn.count('id').alias('nb')).collect()]
#Gathering them all:
list_L0 = [list_beef0[0], list_chicken0[0], list_shrimp0[0], list_fish0[0]]

#Name of the modalities:
modalities = ['Beef', 'Chicken', 'Shrimp', 'Fish']

In [172]:
#Creating the totals:
totals = [i+j for i,j in zip(list_L1,list_L0)]
list_L1p = [i / j * 100 for i,j in zip(list_L1, totals)]
list_L0p = [i / j * 100 for i,j in zip(list_L0, totals)]
p=range(len(list_L1)) 

#Plot options:
barWidth = 0.7
plt.figure(figsize=(12,5))
plt.bar(p, list_L1p, color='#C7CF00', edgecolor='white', width=barWidth)
plt.bar(p, list_L0p, bottom=list_L1p, color='crimson', edgecolor='white', width=barWidth)
plt.xlabel("Food inside the burrito")
plt.ylabel('Proportion of good and bad grades')
plt.xticks(p, modalities, fontsize='small', size=12)
plt.gca().legend(('Good','Bad'))
display(plt.show())

We can drop the variable `burrito_forstat`:

In [174]:
df = df.drop('burrito_forstat')

### 4.3 Correlation

In [176]:
for c in df.columns:
  matrix = df.corr(c,'good_grade' )
  print(c,":",np.round(matrix,2))

We can first erase the variables for which the correlation does not exist or equals 0

In [178]:

df = df.drop('Sour cream', 'Fries', 'Queso', 'None', 'taco_villa')

In [179]:
display(df)

## 5 Modeling

In [181]:
###(Bruno): on devra supprimer cette partie à la toute fin et vérifier que tout se run bien, c'est juste pour gagner du temps:

In [182]:
df_path = '/FileStore/tables/Burrito_cleaned_v2.csv'

#Loading the Burrito csv file
df = spark.read.format('csv').options(header='true', inferSchema='true').load(df_path)

#number of observations:
N = df.count()

In [183]:
display(df.limit(5))

id,Chips,Cost,Hunger,Beef,Pico,Guac,Cheese,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,california,carne_asada,carnitas,california_everything,local,surf_and_turf,al_pastor,adobada,surfin_california,utc,miramar,north_park,clairemont,linda_vista,la_jolla,encinitas,carlsbad,oceanside,hillcrest,san_marcos,pacific_beach,downtown,university_heights,kearny_mesa,carmel_valley,lucha_libre_north_park,california_burritos,rigoberto_s_taco_shop,taco_stand,los_tacos,lolita_s_taco_shop,vallarta_express,los_primos_mexican_food,el_zarape,valentine_s_mexican_food,valentines_mexican_food,tony_s_fresh_mexican_food,cancun_mexican_&_seafood,lupe_s_taco_shop,length_na,length_25,length_50,circum_na,circum_25,circum_50,volume_na,volume_25,volume_50,yelp_na,yelp_25,yelp_50,google_na,google_25,google_50,neg,pos,compound,good_grade
26,0,-0.0451011762890213,0.005724215453194856,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,-0.23685624780129297,3.847466564841965,2.315212563527023,0
29,0,-0.38643616257794405,1.241567654322525,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,-0.23685624780129297,1.4584755888987595,0.6255548148108032,0
65,0,0.550561838999491,-1.8480409428508004,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1.5395372921708859,-0.41943771557400883,-1.209812761687522,0
191,1,-0.7210783059984567,0.6236459348878599,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,-0.23685624780129297,-0.41943771557400883,-0.256936345541834,0
418,0,-0.7143854631300466,-3.0838843817201305,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,-0.23685624780129297,-0.41943771557400883,-0.256936345541834,0


### 5.1 Last checks:

Checking for NAs:

In [186]:
na_per_column = df.select([fn.count(fn.when(df[c].isNull(), c)).alias(c) for c in df.columns])
display(na_per_column)

id,Chips,Cost,Hunger,Beef,Pico,Guac,Cheese,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,california,carne_asada,carnitas,california_everything,local,surf_and_turf,al_pastor,adobada,surfin_california,utc,miramar,north_park,clairemont,linda_vista,la_jolla,encinitas,carlsbad,oceanside,hillcrest,san_marcos,pacific_beach,downtown,university_heights,kearny_mesa,carmel_valley,lucha_libre_north_park,california_burritos,rigoberto_s_taco_shop,taco_stand,los_tacos,lolita_s_taco_shop,vallarta_express,los_primos_mexican_food,el_zarape,valentine_s_mexican_food,valentines_mexican_food,tony_s_fresh_mexican_food,cancun_mexican_&_seafood,lupe_s_taco_shop,length_na,length_25,length_50,circum_na,circum_25,circum_50,volume_na,volume_25,volume_50,yelp_na,yelp_25,yelp_50,google_na,google_25,google_50,neg,pos,compound,good_grade
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


No missing or null values :)

In [188]:
assert df.count() == N

### 5.1 Feature selection

#### 5.1.1 Vector Assembler

First, we need to gather all features into one vector for each individual:

In [192]:
#Let's select only features:
features = df.drop('id', 'good_grade').columns

assembler = VectorAssembler(
    inputCols= features,
    outputCol='features')

df = assembler.transform(df)
display(df.select('id', 'features'))

id,features
26,"List(0, 92, List(1, 2, 3, 4, 5, 6, 12, 36, 51, 74, 77, 80, 88, 89, 90, 91), List(-0.0451011762890213, 0.005724215453194856, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.23685624780129297, 3.847466564841965, 2.315212563527023))"
29,"List(0, 92, List(1, 2, 3, 11, 12, 18, 25, 46, 74, 77, 80, 85, 89, 90, 91), List(-0.38643616257794405, 1.241567654322525, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.23685624780129297, 1.4584755888987595, 0.6255548148108032))"
65,"List(0, 92, List(1, 2, 3, 4, 5, 36, 63, 74, 77, 80, 83, 86, 89, 90, 91), List(0.550561838999491, -1.8480409428508004, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.5395372921708859, -0.41943771557400883, -1.209812761687522))"
191,"List(0, 92, List(0, 1, 2, 3, 6, 35, 54, 76, 79, 82, 85, 89, 90, 91), List(1.0, -0.7210783059984567, 0.6236459348878599, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.23685624780129297, -0.41943771557400883, -0.256936345541834))"
418,"List(0, 92, List(1, 2, 41, 69, 76, 79, 82, 83, 86, 89, 90, 91), List(-0.7143854631300466, -3.0838843817201305, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.23685624780129297, -0.41943771557400883, -0.256936345541834))"
222,"List(0, 92, List(1, 2, 74, 77, 80, 89, 90, 91), List(-1.3903625928394818, 0.005724215453194856, 1.0, 1.0, 1.0, -0.23685624780129297, -0.41943771557400883, -0.256936345541834))"
270,"List(0, 92, List(1, 2, 4, 5, 7, 41, 76, 79, 82, 83, 86, 89, 90, 91), List(-0.7210783059984567, 0.6236459348878599, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.23685624780129297, -0.41943771557400883, -0.256936345541834))"
293,"List(0, 92, List(1, 2, 51, 76, 85, 89, 90, 91), List(-0.05179401915743141, 0.005724215453194856, 1.0, 1.0, 1.0, 2.3383182799436106, 1.4644183525205088, 0.7202250419008716))"
243,"List(0, 92, List(1, 2, 6, 28, 30, 45, 62, 79, 85, 88, 89, 90, 91), List(-0.5136001770777391, 0.6236459348878599, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.23685624780129297, -0.41943771557400883, -0.256936345541834))"
278,"List(0, 92, List(1, 2, 3, 4, 5, 6, 35, 57, 68, 76, 79, 82, 85, 88, 89, 90, 91), List(-0.5470643914197902, 0.6236459348878599, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.23685624780129297, -0.41943771557400883, -0.256936345541834))"


The `features` column has the form: [0, number of columns, [columns containing non null values], [values of those columns]]

#### 5.1.2 Chi-Squared feature selection

Let's keep about 50% of the features we have at our disposal. For this we can set the percentile option within the ChiSqSelector method instead of the numTopfeatures option.

In [196]:
selector = ChiSqSelector(selectorType = "percentile", percentile = 0.5, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="good_grade")

df = selector.fit(df).transform(df)
display(df.select('id', 'features', 'selectedFeatures', 'good_grade').limit(5))

id,features,selectedFeatures,good_grade
26,"List(0, 92, List(1, 2, 3, 4, 5, 6, 12, 36, 51, 74, 77, 80, 88, 89, 90, 91), List(-0.0451011762890213, 0.005724215453194856, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.23685624780129297, 3.847466564841965, 2.315212563527023))","List(0, 46, List(0, 1, 2, 7, 45), List(-0.0451011762890213, 1.0, 1.0, 1.0, 1.0))",0
29,"List(0, 92, List(1, 2, 3, 11, 12, 18, 25, 46, 74, 77, 80, 85, 89, 90, 91), List(-0.38643616257794405, 1.241567654322525, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.23685624780129297, 1.4584755888987595, 0.6255548148108032))","List(0, 46, List(0, 1, 6, 7, 12, 23, 42), List(-0.38643616257794405, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",0
65,"List(0, 92, List(1, 2, 3, 4, 5, 36, 63, 74, 77, 80, 83, 86, 89, 90, 91), List(0.550561838999491, -1.8480409428508004, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.5395372921708859, -0.41943771557400883, -1.209812761687522))","List(0, 46, List(0, 1, 2, 32, 41, 43), List(0.550561838999491, 1.0, 1.0, 1.0, 1.0, 1.0))",0
191,"List(0, 92, List(0, 1, 2, 3, 6, 35, 54, 76, 79, 82, 85, 89, 90, 91), List(1.0, -0.7210783059984567, 0.6236459348878599, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.23685624780129297, -0.41943771557400883, -0.256936345541834))","List(0, 46, List(0, 1, 18, 27, 40, 42), List(-0.7210783059984567, 1.0, 1.0, 1.0, 1.0, 1.0))",0
418,"List(0, 92, List(1, 2, 41, 69, 76, 79, 82, 83, 86, 89, 90, 91), List(-0.7143854631300466, -3.0838843817201305, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.23685624780129297, -0.41943771557400883, -0.256936345541834))","List(0, 46, List(0, 36, 40, 41, 43), List(-0.7143854631300466, 1.0, 1.0, 1.0, 1.0))",0


Again, we can see Spark has its own way of treating sparse matrices, it does not store the zeros within the `features` and `selectedFeatures` columns.

In [198]:
df = df.drop('features', 'selectedFeatures')\
       .withColumnRenamed('scaledFeatures', 'features')\
       .withColumnRenamed('good_grade', 'label')\
       .select('features', 'label')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o4383.select.
: org.apache.spark.sql.AnalysisException: cannot resolve '`features`' given input columns: [Pico, circum_50, Bacon, los_tacos, california_everything, volume_50, pos, Chips, volume_na, california_burritos, cancun_mexican_&_seafood, Cheese, yelp_na, carmel_valley, downtown, carlsbad, carne_asada, pacific_beach, google_na, label, Pineapple, Sauce, taco_stand, length_25, Chicken, Ham, Lettuce, yelp_50, vallarta_express, Taquito, Egg, miramar, encinitas, Sushi, google_25, linda_vista, university_heights, Onion, Pork, Bell peper, al_pastor, north_park, Hunger, Salsa49, surfin_california, utc, valentines_mexican_food, Shrimp, lupe_s_taco_shop, valentine_s_mexican_food, volume_25, clairemont, Mushroom, Chile relleno, Lobster, california, circum_25, Carrots, surf_and_turf, Beef, carnitas, san_marcos, los_primos_mexican_food, local, rigoberto_s_taco_shop, compound, Guac, Avocado, Fish, lucha_libre_north_park, oceanside, id, Cabbage, el_zarape, lolita_s_taco_shop, hillcrest, google_50, neg, Corn, adobada, Rice, length_50, yelp_25, la_jolla, length_na, Beans, kearny_mesa, circum_na, Zucchini, Nopales, Tomato, Cilantro, Cost, tony_s_fresh_mexican_food];;
'Project ['features, label#17197]
+- Project [id#15860, Chips#15861, Cost#15862, Hunger#15863, Beef#15864, Pico#15865, Guac#15866, Cheese#15867, Pork#15868, Chicken#15869, Shrimp#15870, Fish#15871, Rice#15872, Beans#15873, Lettuce#15874, Tomato#15875, Bell peper#15876, Carrots#15877, Cabbage#15878, Sauce#15879, Salsa49#15880, Cilantro#15881, Onion#15882, Taquito#15883, ... 70 more fields]
 +- Project [id#15860, Chips#15861, Cost#15862, Hunger#15863, Beef#15864, Pico#15865, Guac#15866, Cheese#15867, Pork#15868, Chicken#15869, Shrimp#15870, Fish#15871, Rice#15872, Beans#15873, Lettuce#15874, Tomato#15875, Bell peper#15876, Carrots#15877, Cabbage#15878, Sauce#15879, Salsa49#15880, Cilantro#15881, Onion#15882, Taquito#15883, ... 70 more fields]
 +- Project [id#15860, Chips#15861, Cost#15862, Hunger#15863, Beef#15864, Pico#15865, Guac#15866, Cheese#15867, Pork#15868, Chicken#15869, Shrimp#15870, Fish#15871, Rice#15872, Beans#15873, Lettuce#15874, Tomato#15875, Bell peper#15876, Carrots#15877, Cabbage#15878, Sauce#15879, Salsa49#15880, Cilantro#15881, Onion#15882, Taquito#15883, ... 72 more fields]
 +- Project [id#15860, Chips#15861, Cost#15862, Hunger#15863, Beef#15864, Pico#15865, Guac#15866, Cheese#15867, Pork#15868, Chicken#15869, Shrimp#15870, Fish#15871, Rice#15872, Beans#15873, Lettuce#15874, Tomato#15875, Bell peper#15876, Carrots#15877, Cabbage#15878, Sauce#15879, Salsa49#15880, Cilantro#15881, Onion#15882, Taquito#15883, ... 71 more fields]
 +- Relation[id#15860,Chips#15861,Cost#15862,Hunger#15863,Beef#15864,Pico#15865,Guac#15866,Cheese#15867,Pork#15868,Chicken#15869,Shrimp#15870,Fish#15871,Rice#15872,Beans#15873,Lettuce#15874,Tomato#15875,Bell peper#15876,Carrots#15877,Cabbage#15878,Sauce#15879,Salsa49#15880,Cilantro#15881,Onion#15882,Taquito#15883,... 70 more fields] csv

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:120)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.ap

Let's keep only columns we are interested in from now on: `id`, `scaledFeatures`, `good_grade`:

In [200]:
df = df.select('features', 'label')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1014.select.
: org.apache.spark.sql.AnalysisException: cannot resolve '`label`' given input columns: [Lettuce, Carrots, vallarta_express, google_25, carnitas, Corn, yelp_50, Shrimp, google_na, circum_25, oceanside, miramar, google_50, length_25, Pineapple, california_everything, carmel_valley, good_grade, Rice, surf_and_turf, valentine_s_mexican_food, downtown, pos, Chile relleno, volume_25, utc, Fish, lolita_s_taco_shop, Onion, Taquito, volume_na, Pork, Bacon, selectedFeatures, carlsbad, local, san_marcos, pacific_beach, surfin_california, carne_asada, los_primos_mexican_food, features, Sushi, id, lucha_libre_north_park, california_burritos, Cabbage, Hunger, Chips, circum_na, university_heights, Zucchini, circum_50, north_park, length_50, hillcrest, los_tacos, al_pastor, Avocado, lupe_s_taco_shop, Nopales, Chicken, Lobster, Beef, Cilantro, Sauce, Mushroom, volume_50, valentines_mexican_food, tony_s_fresh_mexican_food, kearny_mesa, clairemont, neg, la_jolla, Tomato, Salsa49, Cost, Bell peper, encinitas, compound, rigoberto_s_taco_shop, Pico, el_zarape, yelp_25, linda_vista, california, length_na, cancun_mexican_&_seafood, yelp_na, Egg, adobada, Ham, Guac, Beans, taco_stand, Cheese];;
'Project [features#2456, 'label]
+- Project [id#1048, Chips#1049, Cost#1050, Hunger#1051, Beef#1052, Pico#1053, Guac#1054, Cheese#1055, Pork#1056, Chicken#1057, Shrimp#1058, Fish#1059, Rice#1060, Beans#1061, Lettuce#1062, Tomato#1063, Bell peper#1064, Carrots#1065, Cabbage#1066, Sauce#1067, Salsa49#1068, Cilantro#1069, Onion#1070, Taquito#1071, ... 72 more fields]
 +- Project [id#1048, Chips#1049, Cost#1050, Hunger#1051, Beef#1052, Pico#1053, Guac#1054, Cheese#1055, Pork#1056, Chicken#1057, Shrimp#1058, Fish#1059, Rice#1060, Beans#1061, Lettuce#1062, Tomato#1063, Bell peper#1064, Carrots#1065, Cabbage#1066, Sauce#1067, Salsa49#1068, Cilantro#1069, Onion#1070, Taquito#1071, ... 71 more fields]
 +- Relation[id#1048,Chips#1049,Cost#1050,Hunger#1051,Beef#1052,Pico#1053,Guac#1054,Cheese#1055,Pork#1056,Chicken#1057,Shrimp#1058,Fish#1059,Rice#1060,Beans#1061,Lettuce#1062,Tomato#1063,Bell peper#1064,Carrots#1065,Cabbage#1066,Sauce#1067,Salsa49#1068,Cilantro#1069,Onion#1070,Taquito#1071,... 70 more fields] csv

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:120)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at 

### 5.2 Splitting the dataset

We will split our dataset into a training set called `train` and a testing set called `test`, such that the model will be fitted on `train` which contains 80% of the individuals. The quality of our model will be measured on the test set (which contains the remaining 20% observations).

In [203]:
train, test = df.randomSplit([0.8, 0.2], seed=12345) #we set a seed for the reproduction of our results

Let's check the target variable is balanced across the variables:

In [205]:
def info_data(data):
  size = float(data.select('label').count())
  numPositives = data.select('label').where('label == 1').count()
  prop_ones = (float(numPositives)/float(size))*100
  print(f'The number of ones are {numPositives}/{data.count()}')
  print(f'Percentage of ones are {np.round(prop_ones,2)}%')

print('\n Train set:')
info_data(train)

print('\n Test set:')
info_data(test)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1680.select.
: org.apache.spark.sql.AnalysisException: cannot resolve '`label`' given input columns: [carne_asada, Lettuce, Cost, Taquito, Mushroom, Fish, local, carlsbad, lolita_s_taco_shop, Ham, volume_na, Pineapple, los_tacos, kearny_mesa, Chicken, Beans, carnitas, adobada, google_na, google_25, clairemont, downtown, san_marcos, vallarta_express, Tomato, rigoberto_s_taco_shop, oceanside, hillcrest, Onion, Salsa49, volume_25, Cabbage, circum_na, length_50, google_50, Zucchini, university_heights, Avocado, Nopales, compound, Sushi, length_na, lucha_libre_north_park, valentine_s_mexican_food, Hunger, yelp_25, tony_s_fresh_mexican_food, cancun_mexican_&_seafood, utc, Bell peper, id, al_pastor, Chile relleno, good_grade, length_25, encinitas, california_burritos, Cheese, california, Lobster, yelp_50, Beef, north_park, Egg, miramar, la_jolla, surf_and_turf, Shrimp, Pork, Corn, circum_25, surfin_california, Carrots, Cilantro, california_everything, pos, Chips, Guac, carmel_valley, yelp_na, el_zarape, Sauce, Rice, lupe_s_taco_shop, taco_stand, volume_50, linda_vista, Pico, circum_50, valentines_mexican_food, Bacon, pacific_beach, los_primos_mexican_food, neg];;
'Project ['label]
+- Sample 0.0, 0.8, false, 12345
 +- Sort [id#3721 ASC NULLS FIRST, Chips#3722 ASC NULLS FIRST, Cost#3723 ASC NULLS FIRST, Hunger#3724 ASC NULLS FIRST, Beef#3725 ASC NULLS FIRST, Pico#3726 ASC NULLS FIRST, Guac#3727 ASC NULLS FIRST, Cheese#3728 ASC NULLS FIRST, Pork#3729 ASC NULLS FIRST, Chicken#3730 ASC NULLS FIRST, Shrimp#3731 ASC NULLS FIRST, Fish#3732 ASC NULLS FIRST, Rice#3733 ASC NULLS FIRST, Beans#3734 ASC NULLS FIRST, Lettuce#3735 ASC NULLS FIRST, Tomato#3736 ASC NULLS FIRST, Bell peper#3737 ASC NULLS FIRST, Carrots#3738 ASC NULLS FIRST, Cabbage#3739 ASC NULLS FIRST, Sauce#3740 ASC NULLS FIRST, Salsa49#3741 ASC NULLS FIRST, Cilantro#3742 ASC NULLS FIRST, Onion#3743 ASC NULLS FIRST, Taquito#3744 ASC NULLS FIRST, ... 70 more fields], false
 +- Relation[id#3721,Chips#3722,Cost#3723,Hunger#3724,Beef#3725,Pico#3726,Guac#3727,Cheese#3728,Pork#3729,Chicken#3730,Shrimp#3731,Fish#3732,Rice#3733,Beans#3734,Lettuce#3735,Tomato#3736,Bell peper#3737,Carrots#3738,Cabbage#3739,Sauce#3740,Salsa49#3741,Cilantro#3742,Onion#3743,Taquito#3744,... 70 more fields] csv

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:120)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.Curre

As the proportion of \\(goodgrade = 1\\) approximates 50% within the training set, we can assert that this one is pretty balance regarding the distribution of the target variable.

### 5.4 Logistic Regression Model

Here is the loss function we aim to minimize in order to find the coefficients and predict as best as possible whether a customer likes the burrito or not:
$$
\mathcal{L}(w;x,y) = \mathrm{ln}(1+\mathrm{exp}(-yw^Tx)) ~ , ~ y = goodgrade
$$

After minimization of the loss, we end up with the estimated weights:
$$\widehat{w}$$

Then, we just need to set features values in order to predict a probability for an individual, applying the logistic function:

$$\forall i \in \\{1,...,N\\} ~ , ~ f(\widehat{w},x) = \dfrac{1}{1+e^{-\widehat{w}^Tx}}$$

However, the aim is to predict either 1 (the customer likes the burrito) or 0 (the customer dislikes the burrito). Then we have to compare the predicted probability to a specific threshold in order to assign either 1 or 0:

$$\forall i \in \\{1,...,N\\} ~ , ~ \forall threshold \in [0,1] ~, ~ \widehat{goodgrade_i} = \begin{cases} 1 ~ \mathrm{if} ~ f(\widehat{w}, x_i) > threshold \cr
0 ~ \mathrm{otherwise} \cr
\end{cases}$$

#### 5.4.1 Fitting the model

In [210]:
#Initialize the model,
lr = LogisticRegression(labelCol='label', featuresCol='features', maxIter=50)

#Fit on the train set:
logit_model = lr.fit(train)

# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
#trainingSummary = logit_model.summary

#Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
lr.setThreshold(bestThreshold)

#Get predictions for both train and test sets:
predict_train_logit = logit_model.transform(train)
predict_test_logit = logit_model.transform(test)

#Show up 10 first rows:
predict_test_logit.select('label', 'prediction', 'probability').show(10)

#### 5.4.2 Performance Evaluation

To measure the performance we have to compute those metrics:

-TP: customer who likes the burrito and who was predicted well

-TN: customer who dislikes the burrito and who was predicted well

-FP: customer who dislikes the burrito and who was predicted wrong

-FN: customer who likes the burrito and who was predicted wrong

-P = TP+FN :total number of positives

-N = TN+FP :total number of negatives

In order to get performance metrics about our predictions, we will store into a single **class** called `binary_metrics`, the following metrics:

-accuracy = \\(\dfrac{\mathrm{TP+TN}}{\mathrm{TP+TN+FP+FN}}\\)

-precision = \\(\dfrac{\mathrm{TP}}{\mathrm{TP+FP}}\\)

-recall = \\(\dfrac{\mathrm{TP}}{\mathrm{TP+FN}}\\)

-F1 Score = \\(2 \times \dfrac{\mathrm{precision} \times \mathrm{recall}}{\mathrm{precision} + \mathrm{recall}}\\)
 
-AUC (Area Under the Curve) = \\(\displaystyle \int_0^1 \dfrac{\mathrm{TP}}{\mathrm{P}} d(\dfrac{\mathrm{FP}}{\mathrm{N}}) \\)

In [213]:
class binary_metrics():
  
  def __init__(self, data):
    #Store data into self:
    self.data = data
    
    #Prediction metrics
    self.predictionAndLabels = data.select('prediction', 'label').rdd.map(lambda x : (float(x[0]), float(x[1])))
    self.metrics = MulticlassMetrics(self.predictionAndLabels)
    
    #ROC metrics:
    self.scoreAndLabels = data.select('probability', 'label').rdd.map(lambda x : (float(x[0][1]), float(x[1])))
    self.roc = BinaryClassificationMetrics(self.scoreAndLabels)
    
  def accuracy(self):
    accuracy = self.predictionAndLabels.filter(lambda x: x[0] == x[1]).count()/self.predictionAndLabels.count()
    return np.round(accuracy,2)
  
  def precision(self):
    precision = self.metrics.precision(label = 1.0)
    return np.round(precision,2)
  
  def recall(self):
    recall = self.metrics.recall(label = 1.0)
    return np.round(recall,2)
  
  def fmeasure(self):
    fmeasure = self.metrics.fMeasure(label = 1.0)
    return np.round(fmeasure,2)
  
  def confusion_matrix(self):
    confusion = pd.DataFrame(self.metrics.confusionMatrix().toArray())
    return confusion
  
  def AUC(self):
    auc = self.roc.areaUnderROC
    return np.round(auc,2)

In [214]:
#Get results from the class for predictions from the test set:
logit_test_results = binary_metrics(predict_test_logit)
logit_train_results = binary_metrics(predict_train_logit)

print('\n Train set:')
print('Confusion matrix: \n')
print(logit_train_results.confusion_matrix(), "\n")

print('Accuracy =',logit_train_results.accuracy())
print('Precision =',logit_train_results.precision())
print('Recall =',logit_train_results.recall())
print('F1 Score =',logit_train_results.fmeasure())
print('Area Under the Curve =',logit_train_results.AUC())


print('\n Test set:')
print('Confusion matrix: \n')
print(logit_test_results.confusion_matrix(), "\n")

print('Accuracy =',logit_test_results.accuracy())
print('Precision =',logit_test_results.precision())
print('Recall =',logit_test_results.recall())
print('F1 Score =',logit_test_results.fmeasure())
print('Area Under the Curve =',logit_test_results.AUC())

Let's compute try another logistic model with K-Fold Cross Validation:

#### 5.4.3 K-Fold Cross Validation

In [217]:
#Let's set the parameter grid for hyper-parameter optimization:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.aggregationDepth,[2,5,10])\
    .addGrid(lr.elasticNetParam,[0.5, 1.0])\
    .addGrid(lr.fitIntercept,[False, True])\
    .addGrid(lr.maxIter,[10, 50])\
    .addGrid(lr.regParam,[0, 0.5]) \
    .build()

evaluator= BinaryClassificationEvaluator(rawPredictionCol= 'rawPrediction', labelCol='label')

# Create 5-fold CrossValidator
cv_logit = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=4)

# Run cross validations
cvModel_logit = cv_logit.fit(train)

# this will likely take a fair amount of time because of the amount of models that we're creating and testing
predict_train_logit_k = cvModel_logit.transform(train)
predict_test_logit_k = cvModel_logit.transform(test)
print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(predict_train_logit_k)))
print("The area under ROC for test set after CV  is {}".format(evaluator.evaluate(predict_test_logit_k)))

Hyper-parameter optimization does not seem efficient as the AUC has not been affected.

### 5.5 Decision Tree classifier

First we have to set the label and features indexers to set the data in a good format for the decision tree model:

In [221]:
#Label indexer:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)

#Feature indexer (maxCategories = 2, such that every column with categories > 2 will be treated as a continuous variable):
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1546.fit.
: java.lang.IllegalArgumentException: Field "label" does not exist.
Available fields: id, Chips, Cost, Hunger, Beef, Pico, Guac, Cheese, Pork, Chicken, Shrimp, Fish, Rice, Beans, Lettuce, Tomato, Bell peper, Carrots, Cabbage, Sauce, Salsa49, Cilantro, Onion, Taquito, Pineapple, Ham, Chile relleno, Nopales, Lobster, Egg, Mushroom, Bacon, Sushi, Avocado, Corn, Zucchini, california, carne_asada, carnitas, california_everything, local, surf_and_turf, al_pastor, adobada, surfin_california, utc, miramar, north_park, clairemont, linda_vista, la_jolla, encinitas, carlsbad, oceanside, hillcrest, san_marcos, pacific_beach, downtown, university_heights, kearny_mesa, carmel_valley, lucha_libre_north_park, california_burritos, rigoberto_s_taco_shop, taco_stand, los_tacos, lolita_s_taco_shop, vallarta_express, los_primos_mexican_food, el_zarape, valentine_s_mexican_food, valentines_mexican_food, tony_s_fresh_mexican_food, cancun_mexican_&_seafood, lupe_s_taco_shop, length_na, length_25, length_50, circum_na, circum_25, circum_50, volume_na, volume_25, volume_50, yelp_na, yelp_25, yelp_50, google_na, google_25, google_50, neg, pos, compound, good_grade, features, selectedFeatures
	at org.apache.spark.sql.types.StructType$$anonfun$apply$1.apply(StructType.scala:274)
	at org.apache.spark.sql.types.StructType$$anonfun$apply$1.apply(StructType.scala:274)
	at scala.collection.MapLike$class.getOrElse(MapLike.scala:128)
	at scala.collection.AbstractMap.getOrElse(Map.scala:59)
	at org.apache.spark.sql.types.StructType.apply(StructType.scala:273)
	at org.apache.spark.ml.feature.StringIndexerBase$class.validateAndTransformSchema(StringIndexer.scala:86)
	at org.apache.spark.ml.feature.StringIndexer.validateAndTransformSchema(StringIndexer.scala:110)
	at org.apache.spark.ml.feature.StringIndexer.transformSchema(StringIndexer.scala:153)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:74)
	at org.apache.spark.ml.feature.StringIndexer.fit(StringIndexer.scala:136)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 IllegalArgumentException Traceback (most recent call last)
 <command-2427207629766061> in <module> 
 1 #Label indexer: 
 ----> 2 labelIndexer = StringIndexer ( inputCol = "label" , outputCol = "indexedLabel" ) . fit ( df ) 
 3 
 4 #Feature indexer (maxCategories = 2, such that every column with categories > 2 will be treated as a continuous variable): 
 5 featureIndexer = \ 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a lis

In [222]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Train model. This also runs the indexers.
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

dt_model = pipeline.fit(train)

# Make predictions:
predict_train_dt = dt_model.transform(train)
predict_test_dt = dt_model.transform(test)

In [223]:
#Get results from the class for predictions from the test set:
dt_test_results = binary_metrics(predict_test_dt)
dt_train_results = binary_metrics(predict_train_dt)

print('\n Train set:')
print('Confusion matrix: \n')
print(dt_train_results.confusion_matrix(), "\n")

print('Accuracy =',dt_train_results.accuracy())
print('Precision =',dt_train_results.precision())
print('Recall =',dt_train_results.recall())
print('F1 Score =',dt_train_results.fmeasure())
print('Area Under the Curve =',dt_train_results.AUC())


print('\n Test set:')
print('Confusion matrix: \n')
print(dt_test_results.confusion_matrix(), "\n")

print('Accuracy =',dt_test_results.accuracy())
print('Precision =',dt_test_results.precision())
print('Recall =',dt_test_results.recall())
print('F1 Score =',dt_test_results.fmeasure())
print('Area Under the Curve =',dt_test_results.AUC())

### 5.6 Random Forest classifier

In [225]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df)

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
rf_model = pipeline.fit(train)

# Make predictions.
predict_train_rf = rf_model.transform(train)
predict_test_rf = rf_model.transform(test)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o4624.fit.
: java.lang.IllegalArgumentException: Field "label" does not exist.
Available fields: id, Chips, Cost, Hunger, Beef, Pico, Guac, Cheese, Pork, Chicken, Shrimp, Fish, Rice, Beans, Lettuce, Tomato, Bell peper, Carrots, Cabbage, Sauce, Salsa49, Cilantro, Onion, Taquito, Pineapple, Ham, Chile relleno, Nopales, Lobster, Egg, Mushroom, Bacon, Sushi, Avocado, Corn, Zucchini, california, carne_asada, carnitas, california_everything, local, surf_and_turf, al_pastor, adobada, surfin_california, utc, miramar, north_park, clairemont, linda_vista, la_jolla, encinitas, carlsbad, oceanside, hillcrest, san_marcos, pacific_beach, downtown, university_heights, kearny_mesa, carmel_valley, lucha_libre_north_park, california_burritos, rigoberto_s_taco_shop, taco_stand, los_tacos, lolita_s_taco_shop, vallarta_express, los_primos_mexican_food, el_zarape, valentine_s_mexican_food, valentines_mexican_food, tony_s_fresh_mexican_food, cancun_mexican_&_seafood, lupe_s_taco_shop, length_na, length_25, length_50, circum_na, circum_25, circum_50, volume_na, volume_25, volume_50, yelp_na, yelp_25, yelp_50, google_na, google_25, google_50, neg, pos, compound, good_grade
	at org.apache.spark.sql.types.StructType$$anonfun$apply$1.apply(StructType.scala:274)
	at org.apache.spark.sql.types.StructType$$anonfun$apply$1.apply(StructType.scala:274)
	at scala.collection.MapLike$class.getOrElse(MapLike.scala:128)
	at scala.collection.AbstractMap.getOrElse(Map.scala:59)
	at org.apache.spark.sql.types.StructType.apply(StructType.scala:273)
	at org.apache.spark.ml.feature.StringIndexerBase$class.validateAndTransformSchema(StringIndexer.scala:86)
	at org.apache.spark.ml.feature.StringIndexer.validateAndTransformSchema(StringIndexer.scala:110)
	at org.apache.spark.ml.feature.StringIndexer.transformSchema(StringIndexer.scala:153)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:74)
	at org.apache.spark.ml.feature.StringIndexer.fit(StringIndexer.scala:136)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 IllegalArgumentException Traceback (most recent call last)
 <command-2427207629766089> in <module> 
 1 # Index labels, adding metadata to the label column. 
 2 # Fit on whole dataset to include all labels in index. 
 ----> 3 labelIndexer = StringIndexer ( inputCol = "label" , outputCol = "indexedLabel" ) . fit ( df ) 
 4 
 5 # Automatically identify categorical features, and index them. 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of para

In [226]:
#Get results from the class for predictions from the test set:
rf_test_results = binary_metrics(predict_train_rf)
rf_train_results = binary_metrics(predict_test_rf)

print('\n Train set:')
print('Confusion matrix: \n')
print(rf_train_results.confusion_matrix(), "\n")

print('Accuracy =',rf_train_results.accuracy())
print('Precision =',rf_train_results.precision())
print('Recall =',rf_train_results.recall())
print('F1 Score =',rf_train_results.fmeasure())
print('Area Under the Curve =',rf_train_results.AUC())


print('\n Test set:')
print('Confusion matrix: \n')
print(rf_test_results.confusion_matrix(), "\n")

print('Accuracy =',rf_test_results.accuracy())
print('Precision =',rf_test_results.precision())
print('Recall =',rf_test_results.recall())
print('F1 Score =',rf_test_results.fmeasure())
print('Area Under the Curve =',rf_test_results.AUC())

### 5.7 Gradient-boosted tree classifier

In [228]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df)

# Train a GBT model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)

# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model.  This also runs the indexers.
model = pipeline.fit(train)

# Make predictions.
predict_train_gbt = gbt_model.transform(train)
predict_test_gbt = gbt_model.transform(test)


gbtModel = model.stages[2]
print(gbtModel)  # summary only

In [229]:
#Get results from the class for predictions from the test set:
gbt_test_results = binary_metrics(predict_train_gbt)
gbt_train_results = binary_metrics(predict_test_gbt)

print('\n Train set:')
print('Confusion matrix: \n')
print(gbt_train_results.confusion_matrix(), "\n")

print('Accuracy =',gbt_train_results.accuracy())
print('Precision =',gbt_train_results.precision())
print('Recall =',gbt_train_results.recall())
print('F1 Score =',gbt_train_results.fmeasure())
print('Area Under the Curve =',gbt_train_results.AUC())


print('\n Test set:')
print('Confusion matrix: \n')
print(gbt_test_results.confusion_matrix(), "\n")

print('Accuracy =',gbt_test_results.accuracy())
print('Precision =',gbt_test_results.precision())
print('Recall =',gbt_test_results.recall())
print('F1 Score =',gbt_test_results.fmeasure())
print('Area Under the Curve =',gbt_test_results.AUC())

### 6. Determining the best burrito

In [231]:
df_path = '/FileStore/tables/df_withall.csv'

#Loading the Burrito csv file
df_withall = spark.read.format('csv').options(header='true', inferSchema='true').load(df_path)


In [232]:
display(df_withall)

Here as a grade depend to the reviewer that give it and his severerity, we decided to reshape it with the smooth average of the reviwer's grade. Here the aim is to take into account the average grade gave by the reviewer, but also to take into account the number of review that he gave. If the reviewer gave only 1 review of 2 the score of the review shouldn't be as important as one that gave 10 review of 2. Because for the one that gave only one review of 2 it can be that it was justified and a very bad burrito.

In [234]:
window_spec = Window.partitionBy('Reviewer')
avg_grade = fn.mean('overall').over(window_spec)
n = fn.count('id').over(window_spec)

df_withall= df_withall.withColumn('avg_grade', avg_grade)
df_withall= df_withall.withColumn('n', n)

alpha = 5
meanrevw = df_withall.agg({"overall" : 'avg'}).first()[0]
smoothed = fn.udf(lambda x,n: (n*x + meanrevw*alpha)/(n + alpha))
df_withall = df_withall.withColumn("avg_smooth", smoothed(df_withall.avg_grade, df_withall.n))

meanrevwr = df_withall.select('Reviewer', 'avg_smooth').distinct().agg(fn.mean('avg_smooth')).first()[0]
centr = fn.udf(lambda x: x-meanrevw)
df_withall = df_withall.withColumn("avg_grade_scaled",centr(df_withall.avg_smooth))
df_withall = df_withall.drop('avg_grade','n','avg_smooth')

df_withall = df_withall.withColumn('score', df_withall.overall -df_withall.avg_grade_scaled)

df_withall.select('id','Reviewer','score').show(5)


In [235]:
compo = df_withall.columns[10:44]
compo.append('id')
compo.append('score')
df_bis = df_withall.select(compo)
display(df_bis)

In [236]:
#Delete variables that do not give any informations
to_drop = []
for var in df_bis.columns:
  if df_bis.agg({var : 'sum'}).first()[0]==0:
    to_drop.append(var)
print(to_drop)
df_bis.drop('Queso')

In [237]:
ingredient = df_bis.columns[0:33]
for each in ingredient: 
  others = ingredient
  others.remove(each)
  for each2 in others:
    interact = fn.when(df_bis[each]== 0, 0).when(df_bis[each2]== 0, 0).otherwise(1)
    df_bis = df_bis.withColumn(str(each)+'_'+str(each2), interact)

In [238]:
features = df_bis.drop('id', 'score').columns

assembler = VectorAssembler(
    inputCols= features,
    outputCol='features')

df_bis = assembler.transform(df_bis)
display(df_bis.select('id', 'features'))


We have still too much information regarding the ingredient and combinaison of ingredients. Here, we have more variables than observations. So we decided to keep only the correlated to score ones as only master ingredients are interesting for us.

In [240]:
selector = ChiSqSelector(selectorType = "percentile", percentile = 0.2, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="score")

df_bis = selector.fit(df_bis).transform(df_bis)
display(df_bis.select('id', 'features', 'selectedFeatures', 'score').limit(5))

In [241]:
lr = LinearRegression(labelCol='score', featuresCol='selectedFeatures', maxIter=50)

#Fit on the train set:
linear_model = lr.fit(df_bis)


#Get predictions for both train and test sets:
prediction = linear_model.transform(df_bis)

#Show up 10 first rows:
prediction.select('SelectedFeatures','prediction').show(10)

In [242]:
print(linear_model.coefficients)
print(linear_model.intercept)

In [243]:
print(linear_model.explainParams)

In [244]:
coef = np.array([i for i in linear_model.coefficients])
coef_pos = np.where(coef > 0)
coef_pos[0]